In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
import data_processing as dp
from scipy import signal
from scipy.stats import skew
import seaborn as sns
import matplotlib.pyplot as plt
import pywt
import os
import time
import h5py

In [2]:
for dirname, _, filenames in os.walk('.\data'):
    for filename in filenames:
        print(os.path.join(dirname,filename))

.\data\G04_FoG_trial_1_emg.csv
.\data\G04_FoG_trial_2_emg.csv
.\data\G06_FoG_trial_1_emg.csv
.\data\G06_FoG_trial_2_emg.csv
.\data\G06_FoG_trial_3_emg.csv
.\data\G07_Freezing_Trial1_trial_1_emg.csv
.\data\G08_FoG_1_trial_1_emg.csv
.\data\G08_FoG_2_trial_1_emg.csv
.\data\G11_FoG_trial_1_emg.csv
.\data\G11_FoG_trial_2_emg.csv
.\data\P379_M050_2_OFF_A_FoG_trial_1_emg.csv
.\data\P379_M050_2_OFF_A_FoG_trial_2_emg.csv
.\data\P379_M050_2_OFF_A_FoG_trial_3_emg.csv
.\data\P379_M050_2_OFF_B_FoG_trial_1_emg.csv
.\data\P379_M050_2_OFF_B_FoG_trial_2_emg.csv
.\data\P379_M050_2_OFF_B_FoG_trial_3_emg.csv
.\data\P551_M050_2_A_FoG_trial_1_emg.csv
.\data\P551_M050_2_B_FoG_trial_1_emg.csv
.\data\P551_M050_2_B_FoG_trial_2_emg.csv
.\data\P812_M050_2_B_FoG_trial_1_emg.csv
.\data\P812_M050_2_B_FoG_trial_2_emg.csv
.\data\其他\labels.txt
.\data\其他\P812_M050_2_B_FoG_trial1_annotation.csv
.\data\其他\P812_M050_2_B_FoG_trials.mat
.\data\其他\P812_M050_2_B_FoG_trial_1_out_left_foot.csv
.\data\其他\P812_M050_2_B_FoG_trial_1

In [72]:
path2 = '.\data\正常\G11_Walking_trial_4_emg.csv'

In [4]:
path = './processed data/data_set_after_window_withoutSC.hdf5'
with h5py.File(path,'r') as f:
    x = f['cwt_data'][...]
    y = f['label2'][...]

In [ ]:
import imp
imp.reload(dp)

In [5]:
import ctypes
player = ctypes.windll.kernel32
player.Beep(1000,200)

1

In [6]:
feature = dp.generate_feature(x,50,0,bins=9,ranges=(-70,70))
#feature2 = dp.generate_feature(x2)
player.Beep(1000,200)

threshold_WAMP:50.0, threshold_ZC:0.0, threshold_SSC:0.0,bins:9,ranges:(-70,70)
IEMG,MAV,SSI,VAR,RMS,WL,ZC,SSC,WAMP,skew,Acti,AR,HIST


1

In [73]:
feature2,y2 = dp.pipeline_feature(path2,width=256,scaler=False,threshold_WAMP=50,
                                  bins=9,ranges=(-70,70))

threshold_WAMP:50.0, threshold_ZC:0.0, threshold_SSC:0.0,bins:9,ranges:(-70,70)
IEMG,MAV,SSI,VAR,RMS,WL,ZC,SSC,WAMP,skew,Acti,AR,HIST


In [74]:
sc = StandardScaler(with_mean=False)
feature_sc = sc.fit_transform(feature)
feature2_sc = sc.fit_transform(feature2)

In [20]:
from tensorflow.keras import layers,Model,callbacks
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import OneHotEncoder

In [42]:
#ind = ((y==1)|(y==2)|(y==3)|(y==6))
ind = ((y==4)|(y==1)|(y==2)|(y==3)|(y==6))
ind_f = [0,1,6,42,46,57,62]
#y_01 = y[ind].copy()
#y_01[y_01==1]=0
#y_01[y_01==2]=1
#y_01[y_01==3]=2
#y_01[y_01==6]=3
y_01 = y.copy()
y_01[ind] = 1
oh_ec = OneHotEncoder()
y_oh = oh_ec.fit_transform(y_01[:,np.newaxis]).toarray()
x_full,x_test,y_full,y_test = train_test_split(feature_sc[:-2000],y_01[:-2000],test_size=0.2,random_state=123)
x_train,x_valid,y_train,y_valid = train_test_split(x_full,y_full,test_size=0.2,random_state=555)

In [66]:
input_ = layers.Input(shape=feature.shape[1:])
l1 = layers.Dense(128,activation='tanh')(input_)
drop1 = layers.Dropout(0.2)(l1)
l2 = layers.Dense(64,activation='tanh')(drop1)
drop2 = layers.Dropout(0.2)(l2)
#l3 = layers.Dense(32,activation='tanh')(l2)
#l4 = layers.Dense(16,activation='tanh')(l3)
output = layers.Dense(1,activation='sigmoid')(drop2)
model = Model(inputs=[input_],outputs=[output])

In [67]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [68]:
early_stopping = callbacks.EarlyStopping(patience = 20,
                                         monitor = 'val_accuracy', 
                                         restore_best_weights=True)
history = model.fit(x_train,y_train,validation_data=[x_valid,y_valid],
                    epochs=100,batch_size=50,
                   callbacks=[early_stopping])

Train on 27219 samples, validate on 6805 samples
Epoch 1/100
27219/27219 [==============================] - 2s 84us/sample - loss: 0.3941 - accuracy: 0.8306 - val_loss: 0.3529 - val_accuracy: 0.8573
Epoch 2/100
27219/27219 [==============================] - 2s 58us/sample - loss: 0.3183 - accuracy: 0.8729 - val_loss: 0.3373 - val_accuracy: 0.8679
Epoch 3/100
27219/27219 [==============================] - 1s 51us/sample - loss: 0.2959 - accuracy: 0.8825 - val_loss: 0.2785 - val_accuracy: 0.8918
Epoch 4/100
27219/27219 [==============================] - 1s 50us/sample - loss: 0.2805 - accuracy: 0.8908 - val_loss: 0.2586 - val_accuracy: 0.8977
Epoch 5/100
27219/27219 [==============================] - 1s 52us/sample - loss: 0.2677 - accuracy: 0.8953 - val_loss: 0.2359 - val_accuracy: 0.9108
Epoch 6/100
27219/27219 [==============================] - 1s 51us/sample - loss: 0.2544 - accuracy: 0.9006 - val_loss: 0.2246 - val_accuracy: 0.9154
Epoch 7/100
27219/27219 [==========================

Epoch 55/100
27219/27219 [==============================] - 1s 49us/sample - loss: 0.1002 - accuracy: 0.9634 - val_loss: 0.1143 - val_accuracy: 0.9580
Epoch 56/100
27219/27219 [==============================] - 1s 50us/sample - loss: 0.0963 - accuracy: 0.9649 - val_loss: 0.1061 - val_accuracy: 0.9630
Epoch 57/100
27219/27219 [==============================] - 1s 48us/sample - loss: 0.0974 - accuracy: 0.9645 - val_loss: 0.1048 - val_accuracy: 0.9637
Epoch 58/100
27219/27219 [==============================] - 1s 48us/sample - loss: 0.0935 - accuracy: 0.9646 - val_loss: 0.1034 - val_accuracy: 0.9616
Epoch 59/100
27219/27219 [==============================] - 1s 49us/sample - loss: 0.0965 - accuracy: 0.9648 - val_loss: 0.1092 - val_accuracy: 0.9618
Epoch 60/100
27219/27219 [==============================] - 1s 49us/sample - loss: 0.0950 - accuracy: 0.9652 - val_loss: 0.1263 - val_accuracy: 0.9564
Epoch 61/100
27219/27219 [==============================] - 1s 48us/sample - loss: 0.0875 - ac

In [69]:
y_pred=model.predict(x_test)
test = metrics.accuracy_score(y_test,y_pred>0.5)
y_pred=model.predict(x_valid)
valid = metrics.accuracy_score(y_valid,y_pred>0.5)
print('test:%f'%test)
print('valid:%f'%valid)

test:0.974606
valid:0.970610


In [70]:
y_pred=model.predict(feature_sc[41872:])
metrics.accuracy_score(y_01[41872:],y_pred>0.5)

0.8476297968397292

In [48]:
set(y_01)

{0, 1}

In [75]:
y_pred=model.predict(feature2_sc)
metrics.accuracy_score(y2,y_pred>0.5)

0.6651651651651652